In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json

In [2]:
# helper file - set up the term lookup

termlist = dict()
termlist['hilton'] = ['HiltonHotels', 'HiltonHHonors', 'HiltonWorldwide', 'Hilton']
termlist['marriott'] = ['Marriott', 'Mariott', 'Marriot', 'MarriottIntl']
termlist['starwood'] = ['sheraton', 'spg','starwood']

termlookup = {}
for ii in termlist.keys():
    for jj in termlist[ii]:
        termlookup[jj.lower()] = ii.lower()    # make sure whole dict is lower for searching

term_string = ", ".join(termlookup.keys())
FOLLOW_TERM = term_string    # Creates string for Twitter monitor - check: does case matter?
FOLLOW_list = FOLLOW_TERM.split()
print(FOLLOW_TERM)

marriot, marriottintl, hiltonhhonors, mariott, hiltonworldwide, sheraton, hilton, starwood, marriott, spg, hiltonhotels


In [37]:
# helper function - agg up the tweets

def hashcounter(tweets):    
    hashtags = dict()
    for key, tweet in tweets.items():
        tweethash = tweet.get('hashtags')
        type(tweethash)
        for ii in tweethash:
            if ii.get('text') in hashtags.keys():
                hashtags[ii.get('text')] += 1
            else:
                hashtags[ii.get('text')] = 1
    
    from operator import itemgetter
    for k, v in sorted(hashtags.items(), key=itemgetter(1), reverse=True)[0:10]:
        print k, v

    return hashtags

In [87]:
# a dict of dicts way to load the RAW tweets

filename = 'tweettest.txt'
fp = open(filename)

tweetsraw = dict()
errorlines = []
for ii in range(500):
    try:
        line = fp.readline()
        line = eval(line) # and now the line is a dict
        #out = tweetproc(line, termlookup)
        tweetsraw[ii] = line
    except:
        errorlines.append(ii)
print len(errorlines)

0


In [91]:
# a dict of dicts way to load the PROCESSED tweets!!!
# note: requires all the termlookup and tweetproc initialization

filename = 'tweettest.txt'
fp = open(filename)

tweets = dict()
errorlines = []
for ii in range(500):
    try:
        line = fp.readline()
        line = eval(line) # and now the line is a dict
        out = tweetproc(line, termlookup)
        tweets[ii] = out
    except:
        errorlines.append(ii)
print len(errorlines)
print (errorlines[1])

0


IndexError: list index out of range

In [90]:
# main tweet processing function - line by line, requires term lookup and dict of dict filestructure

def tweetproc(line, termlookup):    # inputs line as a dict
    #print(line.keys()) 
    #print(line['text'])
    tweet = dict()
    tweet['text'] = line.get('text')
    tweet['username'] = line.get('user').get('screen_name')
    tweet['userloc'] = line.get('user').get('location')
    tweet['tweetloc'] = line.get('geo')
    tweet['time'] = line.get('created_at')
    tweet['lang'] = line.get('lang')
    tweet['id'] = line.get('id')
    
    # these here may be lists - need further processing to flatten
    tweet['hashtags'] = line.get('entities').get('hashtags')
    tweet['urls'] = line.get('entities').get('urls')
    tweet['user_mentions'] = line.get('entities').get('user_mentions')
    
    # Something needs to be debugged here!!!
    #tweet['retweeted_id'] = line.get('retweeted_status').get('id')   
    #tweet['retweeted_user'] = line.get('retweeted_status').get('user').get('screen_name')
    
    brands = []
    #print line.get('text').lower()
    for ii in termlookup.keys():
        if line.get('text').lower().find(ii) >= 0:
            brands.append(ii)
    tweet['brands'] = brands
    
    # tweet['err'] = line.get('fake')
    return tweet

In [ ]:
# Documenting tweet stuff
# tweetsraw.get(ii).get('entities').get('hashtags') --> returns a list of dicts / empty lists

In [76]:
isaretweet = 0
for ii in range(100):
    #tweetsraw.get(ii).get('entities').get('hashtags')
    #print type(tweetsraw.get(ii).get('retweeted_status'))
    try:
        tweetsraw[ii]['retweeted_status']
        isaretweet += 1
        #print ii
    except:
        a = 0

print isaretweet

32


In [79]:
#tweetsraw[0]

In [32]:
# a dataframe way to load the raw tweets

filename = 'tweettest.txt'
fp = open(filename)

records = [eval(line) for line in fp]
frame = pd.DataFrame(records)

In [34]:
# exploring the data frame

# two different ways of accessing columns
#print frame['entities'][:5]
#print frame.entities[:5]

# to add columns - assign a scalar or a Series
# use del to delete (but how did this work without function notation?)
# NOTE: returned column is a view, not a copy - operations on that underlying series will modify the dataframe!

# Dict of dicts!  Interpreted as dataframe with outer dict as column and inner keys as rows
framesub = frame[:5]
framesub

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,None,None,Wed Jul 27 02:04:03 +0000 2016,"{u'user_mentions': [{u'indices': [3, 14], u'sc...","{u'media': [{u'source_user_id': 247583674, u'e...",0,False,low,None,758120737500721152,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @ZIPERATIVO: Perez Hilton expõe a Demi Lova...,1469585043679,False,"{u'utc_offset': None, u'statuses_count': 55255..."
1,None,None,Wed Jul 27 02:04:06 +0000 2016,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,758120749634912256,...,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",#nicki hilton nude buffy tyler sexy https://t....,1469585046572,False,"{u'utc_offset': None, u'statuses_count': 8609,..."
2,None,None,Wed Jul 27 02:06:16 +0000 2016,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,758121296341241856,...,NaN,NaN,0,False,NaN,"<a href=""http://dlvr.it"" rel=""nofollow"">dlvr.i...",#toprated Hilton Garden Inn Anaheim Garden Gro...,1469585176917,False,"{u'utc_offset': -25200, u'statuses_count': 300..."
3,None,None,Wed Jul 27 02:06:22 +0000 2016,"{u'user_mentions': [{u'indices': [3, 18], u'sc...",NaN,0,False,low,None,758121321779765248,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""https://about.twitter.com/products/tw...","RT @classifiedfact: In 2011, a man crashed Par...",1469585182982,False,"{u'utc_offset': 19800, u'statuses_count': 682,..."
4,None,None,Wed Jul 27 02:06:22 +0000 2016,"{u'user_mentions': [{u'indices': [3, 18], u'sc...",NaN,0,False,low,None,758121321783922688,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""https://about.twitter.com/products/tw...","RT @classifiedfact: In 2011, a man crashed Par...",1469585182983,False,"{u'utc_offset': -14400, u'statuses_count': 249..."
